# Configure test parameters

Each test or metric in the ValidMind Developer Framework comes with default parameters and values that are configurable for your specific use case.

ValidMind provides a comprehensive suite of default metrics out of the box to evaluate and document your models and datasets. Typically, documenting a specific model involves configuring these tests and metrics with a config to suit your use case.

This interactive notebook provides a step-by-step guide for configuring parameters for a test or set of tests within a specific section of your model documentation, and documenting a simple classification model using ValidMind with a bank customer churn dataset. The bank customer churn dataset used here is from [Kaggle](https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data).

## Contents<a id='toc0_'></a>    
- [About ValidMind](#toc1_)    
  - [Before you begin](#toc1_1_)    
  - [New to ValidMind?](#toc1_2_)    
  - [Key concepts](#toc1_3_)    
- [Install the client library](#toc2_)    
- [Initialize the Python environment](#toc3_)    
- [Initialize the client library](#toc4_)    
  - [Preview the model's documentation template](#toc4_1_)    
- [Load the demo dataset](#toc5_)    
  - [Documenting the model](#toc5_1_)    
    - [Prepocess the raw dataset](#toc5_1_1_)    
  - [Initialize the ValidMind datasets](#toc5_2_)    
  - [Assign predictions to the datasets](#toc5_3_)    
    - [Run the template documentation suite](#toc5_3_1_)    
    - [Configuration of parameters for model diagnosis tests](#toc5_3_2_)    
- [Next steps](#toc6_)    
  - [Work with your model documentation](#toc6_1_)    
  - [Discover more learning resources](#toc6_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_'></a>[About ValidMind](#toc0_)

ValidMind is a platform for managing model risk, including risk associated with AI and statistical models. You use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

### <a id='toc1_1_'></a>[Before you begin](#toc0_)

This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language. 

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

### <a id='toc1_2_'></a>[New to ValidMind?](#toc0_)

If you haven't already seen our [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html), we recommend you explore the available resources for developers at some point. There, you can learn more about documenting models, find code samples, or read our developer reference.

::: {.callout-tip}

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)

:::

### <a id='toc1_3_'></a>[Key concepts](#toc0_)

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Developer Framework, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

**Custom metrics**: Custom metrics are functions that you define to evaluate your model or dataset. These functions can be registered with ValidMind to be used in the platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind framework. They can be any of the following:

  - **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
  - **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
  - **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom metric.
  - **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom metric. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a metric, customize its behavior, or provide additional context.

**Outputs**: Custom metrics can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: the [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.


## <a id='toc2_'></a>[Install the client library](#toc0_)

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [ ]:
%pip install -q validmind

## <a id='toc3_'></a>[Initialize the Python environment](#toc0_)


In [ ]:
import xgboost as xgb

%matplotlib inline

## <a id='toc4_'></a>[Initialize the client library](#toc0_)

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details, making sure to select **Binary classification** as the template and **Marketing/Sales - Attrition/Churn Management** as the use case, and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="..."
)

### <a id='toc4_1_'></a>[Preview the model's documentation template](#toc0_)

All models are assigned a documentation template when registered. The template defines a list of sections that are used to document the model. Each section can contain any number of rich text and test driven blocks that populate the documentation. Test driven blocks are populated by running tests against the model.

We can preview the model documentation template for this model by running the following code:


In [ ]:
vm.preview_template()

## <a id='toc5_'></a>[Load the demo dataset](#toc0_)


For the purpose of this demonstration, we will use a sample dataset provided by the ValidMind library.


In [ ]:
# Import the sample dataset from the library
from validmind.datasets.classification import customer_churn as demo_dataset
# You can try a different dataset with:
# from validmind.datasets.classification import taiwan_credit as demo_dataset

raw_df = demo_dataset.load_data()

### <a id='toc5_1_'></a>[Documenting the model](#toc0_)


We will need to preprocess the dataset and produce the training, test and validation splits first.


#### <a id='toc5_1_1_'></a>[Prepocess the raw dataset](#toc0_)


For demonstration purposes, we simplified the preprocessing using demo_dataset.preprocess which executes the following operations:


In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(raw_df)

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

### <a id='toc5_2_'></a>[Initialize the ValidMind datasets](#toc0_)

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the raw dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset
- `class_labels` — an optional value to map predicted classes to class labels

With all datasets ready, you can now initialize the raw, training and test datasets (`raw_df`, `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):


In [ ]:
vm_dataset = vm.init_dataset(
    dataset=raw_df,
    input_id="raw_dataset",
    target_column=demo_dataset.target_column,
    class_labels=demo_dataset.class_labels
)

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    input_id="test_dataset",
    target_column=demo_dataset.target_column
)

We also initialize a model object using vm.init_model():


In [ ]:
vm_model = vm.init_model(
    model,
    input_id="model"
)

### <a id='toc5_3_'></a>[Assign predictions to the datasets](#toc0_)

We can now use the `assign_predictions()` method from the Dataset object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:


In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
    prediction_values=list(model.predict(x_train))
)
vm_test_ds.assign_predictions(
    model=vm_model,
    prediction_values=list(model.predict(x_val))
)

#### <a id='toc5_3_1_'></a>[Run the template documentation suite](#toc0_)


We are now ready to run the model's documentation tests as defined in its template. The following function runs every test in the template and sends all documentation artifacts to the ValidMind platform.


In [ ]:
full_suite = vm.run_documentation_tests(
    inputs={
        "dataset": vm_train_ds,
        "datasets": (vm_train_ds, vm_test_ds),
        "model": vm_model
    },
    section="model_diagnosis",
)

#### <a id='toc5_3_2_'></a>[Configuration of parameters for model diagnosis tests](#toc0_)

Each test has its default parameters and their values depending on the use case you are trying to solve. ValidMind's developer framework exposes these parameters at the user level so that they can be adjusted based on requirements.

The config can be applied to a specific test to override the default configuration parameters.

The format of the config is:

```
config = {
    "<test1_id>": {
        "<default_param_1>": value,
        "<default_param_2>": value,
    },
     "<test2_id>": {
        "<default_param_1>": value,
        "<default_param_2>": value,
    },
}
```

Users can input the configuration to `run_documentation_tests()` and `run_test_suite()` using **`config`**, allowing fine-tuning the suite according to the specific configuration requirements.


In [ ]:
config = {
    "validmind.model_validation.sklearn.OverfitDiagnosis": {
        "params": {
            "cut_off_percentage": 3,
            "feature_columns": ["Age", "Balance", "Tenure", "NumOfProducts"]
        },
    },
    "validmind.model_validation.sklearn.WeakspotsDiagnosis": {
        "params": {
            "features_columns": ["Age", "Balance"],
            "accuracy_gap_threshold": 85,
        },
    },
    "validmind.model_validation.sklearn.RobustnessDiagnosis": {
        "params": {
            "features_columns": ["Balance", "Tenure"],
            "scaling_factor_std_dev_list": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
            "accuracy_decay_threshold": 4,
        },
    }
}

full_suite = vm.run_documentation_tests(
    inputs={
        "dataset": vm_train_ds,
        "datasets": (vm_train_ds, vm_test_ds),
        "model": vm_model
    },
    section="model_diagnosis",
)

## <a id='toc6_'></a>[Next steps](#toc0_)

You can look at the output produced by the ValidMind Developer Framework right in the notebook where you ran the code, as you would expect. But there is a better way: use the ValidMind platform to work with your model documentation.

### <a id='toc6_1_'></a>[Work with your model documentation](#toc0_)

1. From the [**Model Inventory**](https://app.prod.validmind.ai/model-inventory) in the ValidMind Platform UI, go to the model you registered earlier.

2. Click and expand the **Model Development** section.

What you see is the full draft of your model documentation in a more easily consumable version. From here, you can make qualitative edits to model documentation, view guidelines, collaborate with validators, and submit your model documentation for approval when it's ready. [Learn more ...](https://docs.validmind.ai/guide/working-with-model-documentation.html)

### <a id='toc6_2_'></a>[Discover more learning resources](#toc0_)

We offer many interactive notebooks to help you document models:

- [Run tests & test suites](https://docs.validmind.ai/guide/testing-overview.html)
- [Code samples](https://docs.validmind.ai/guide/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.